# Trying Another (Possible?) Code for Identifying Clusters

First, we import Acqu (which parses the events?) and Timepix (which decodes the data??), as well as numpy, ROOT, and plotting classes of ROOT which allow us to organize and graph the data.

In [1]:
import Acqu as aq
import Timepix
import numpy as np
import ROOT
from rootpy.plotting import Hist, Hist2D, histogram, Canvas
from ipykernel import kernelapp as app

Welcome to JupyROOT 6.16/00


Next, we import the data that was collected and open that data file.


In [2]:
inFile = '/w/work0/mainz/2019_05_Timepix3-Acqu/Timepix_33.dat'
aq.openFile(inFile)

Mk2 Data


Getting the data from timepix A and B:

In [200]:
TimepixAData = []
TimepixBData = []
hitsA = 0
hitsB = 0

def readTimepixData():
    if(aq.epicsEvent==1):
        nHitsA       = aq.getEpicsPV('PPOL:TIMEPIXA:NHITS')  # readTimepixBufferFromFile
        encodedA     = aq.getEpicsPV('PPOL:TIMEPIXA:ENCODED')
        nHitsB       = aq.getEpicsPV('PPOL:TIMEPIXB:NHITS')
        encodedB     = aq.getEpicsPV('PPOL:TIMEPIXB:ENCODED')
        TimepixAData.append(Timepix.Decode(nHitsA,encodedA))
        TimepixBData.append(Timepix.Decode(nHitsB,encodedB))
        global hitsA
        hitsA+=nHitsA
        global hitsB
        hitsB+=nHitsB
        
       
aq.runFunction(readTimepixData,0,50000)
print(hitsA,hitsB)

(65911.0, 46085.0)


I wasn't sure how to access the data the way it was structured, so I attempted to create a new array of data points with the ToA, x position, and y position, and sorted them by ToA.

In [101]:
ToAAData = []
ToAA = []

ToTAData = []
ToTA = []

xAData = []
xA = []

yAData = []
yA = []

dataA = []

for i in range(len(TimepixAData)):
    ToAAData.append(TimepixAData[i]['ToA'])
    ToTAData.append(TimepixAData[i]['ToT'])
    xAData.append(TimepixAData[i]['x'])
    yAData.append(TimepixAData[i]['y'])
    
for j in range(len(ToAAData)):
    for k in range(len(ToAAData[j])):
        ToAA.append(ToAAData[j][k])
        ToTA.append(ToTAData[j][k])
        xA.append(xAData[j][k])
        yA.append(yAData[j][k])
           
for z in range(len(ToAA)):
    dataA.append((ToAA[z],ToTA[z],xA[z],yA[z]))

dataA.sort()

###############################################

ToABData = []
ToAB = []

ToTBData = []
ToTB = []

xBData = []
xB = []

yBData = []
yB = []

dataB = []

for i in range(len(TimepixBData)):
    ToABData.append(TimepixBData[i]['ToA'])
    ToTBData.append(TimepixBData[i]['ToT'])
    xBData.append(TimepixBData[i]['x'])
    yBData.append(TimepixBData[i]['y'])
    
for j in range(len(ToABData)):
    for k in range(len(ToABData[j])):
        ToAB.append(ToABData[j][k])
        ToTB.append(ToTBData[j][k])
        xB.append(xBData[j][k])
        yB.append(yBData[j][k])

for z in range(len(ToAB)):
    dataB.append((ToAB[z],ToTB[z],xB[z],yB[z]))

dataB.sort()

REMINDER: the 0th index is ToA, the 1st is ToT, the 2nd is x position, and the 3rd is y position

At some point we'll need to process/analyze the cluster we have:

In [191]:
def process_clusterA():
    start_index = cl[0]                  # start index of the first hit in the cluster
    min_time = dataA[start_index][0]     # time of the first hit in the cluster
    dtime = 0.0                          # time difference?
    meanx = 0.0
    meany = 0.0
    
    # this next bit was a for loop in Ken's code
    n = start_index
    while(n<start_index+cn):             # go through each hit in the cluster
        if(dataA[n][0] < min_time):      # if we find a time smaller than the current min time
            min_time = dataA[n][0]       # reassign the smaller time to be the min time
        meanx+=dataA[n][2]               # add up all the x positions
        meany+=dataA[n][3]               # add up all the y positions
        n+=1                             # move to the next hit in the cluster

    meanx/=cn                        # calculate the mean x position
    meany/=cn                        # calculate the mean y position
    
    
    tdposA.Fill(meanx,meany)         # plot mean y vs. mean x
    multA.Fill(cn)                   # plot the counts in the cluster
    
    # this next bit was a for loop in Ken's code
    n = start_index
    while(n<start_index+cn):               # for each hit in the cluster
        dtime = dataA[n][0] - min_time     # find time difference between a hit in the cluster and the min time of the cluster
        dthistA.Fill(dtime)                # put the time difference in the histogram
        dtime_toaA.Fill(dtime,dataA[n][1])
        n+=1                               # move to the next hit in the cluster

In [192]:
def process_clusterB():
    start_index = cl[0]                  # start index of the first hit in the cluster
    min_time = dataB[start_index][0]     # time of the first hit in the cluster
    dtime = 0.0                          # time difference?
    meanx = 0.0
    meany = 0.0
    
    # this next bit was a for loop in Ken's code
    n = start_index
    while(n<start_index+cn):             # go through each hit in the cluster
        if(dataB[n][0] < min_time):      # if we find a time smaller than the current min time
            min_time = dataB[n][0]       # reassign the smaller time to be the min time
        meanx+=dataB[n][2]               # add up all the x positions
        meany+=dataB[n][3]               # add up all the y positions
        n+=1                             # move to the next hit in the cluster

    meanx/=cn                        # calculate the mean x position
    meany/=cn                        # calculate the mean y position
    
    
    tdposB.Fill(meanx,meany)         # plot mean y vs. mean x
    multB.Fill(cn)                   # plot the counts in the cluster
    
    # this next bit was a for loop in Ken's code
    n = start_index
    while(n<start_index+cn):               # for each hit in the cluster
        dtime = dataB[n][0] - min_time     # find time difference between a hit in the cluster and the min time of the cluster
        dthistB.Fill(dtime)                # put the time difference in the histogram
        dtime_toaB.Fill(dtime,dataB[n][1])
        n+=1  

Now let's try to identify the clusters.

In [201]:
cl = [None]*100          # a cluster can hold up to 100 hits
cn = 0                   # counts number of hits in current cluster
blen = 0                 # the length of the current tp3 buffer (?)
b = 0                    # the running (current?) buffer position
nc = 0                   # count the number of clusters found

ROOT.enableJSVis()

tdposA = Hist2D(256,0,256,256,0,256)
tdposA.GetXaxis().SetTitle("mean x position of the cluster")
tdposA.GetYaxis().SetTitle("mean y position of the cluster")

multA = Hist(50,0,50)
multA.GetXaxis().SetTitle("number of hits in the cluster")
multA.GetYaxis().SetTitle("number of clusters")

dthistA = Hist(100,0,100)
dthistA.GetXaxis().SetTitle("time diff between a hit and the min time of the cluster")
dthistA.GetYaxis().SetTitle("number of clusters")

dtime_toaA = Hist2D(100,0,100,230,0,230)
dtime_toaA.GetXaxis().SetTitle("time diff between a hit and the min time of the cluster")
dtime_toaA.GetYaxis().SetTitle("ToT")

cA = ROOT.TCanvas("my4","Clusters A", 1000,1000)
cA.Divide(2,2)

blen = hitsA#readTimepixBufferFromFile() - but this has no definition, so I input the number of entries/data points we have
    
while(b<blen):        # for every buffer position in the buffer?
    cn = 0            # zero the cluster counter for the current buffer position
    cl[cn] = b        # assume the current position has the first hit in the cluster and save it#
    cn+=1             # increment the counter, since we've counted a hit
        
    # this next bit was a for loop in Ken's code
    c=b+1             # start at position c that is one above the current buffer position
    while((c<b+99) and (c<blen)):                  # check all positions up to 99 away, as long as we're still in the buffer
        if(abs(dataA[c][0]- dataA[b][0] < 100)):   # if an event at c is within 100ns of that at the current buffer position
            cl[cn] = c                             # save this position to the cluster
            cn+=1                                  # increment the counter, since we've counted a hit
        else:
            process_clusterA()               # analyze the cluster we found
            b+=cn                           # move on beyond this cluster
            cn = 0                          # reset the counter for hits in the cluster
            cl = [None]*100                 # reset the array of cluster hits
            nc+=1                           # increment the number of clusters found
                
            if(c<b+99 and c<blen):          # if we're still in the loop
                cl[cn] = b                  # assume current position is the first in the new cluster
                cn+=1                       # increment the counter, since we've counted a hit
                
        c+=1 # move to the next position and go through this loop again
    b+=1
    
print(nc)

cA.cd(1)
tdposA.Draw("colz")
cA.cd(2)
multA.Draw()
cA.cd(3)
dthistA.Draw()
cA.cd(4)
dtime_toaA.Draw("colz")
cA.Draw()

18684


In [202]:
cl = [None]*100          # a cluster can hold up to 100 hits
cn = 0                   # counts number of hits in current cluster
blen = 0                 # the length of the current tp3 buffer (?)
b = 0                    # the running (current?) buffer position
nc = 0                   # count the number of clusters found

ROOT.enableJSVis()

tdposB = Hist2D(256,0,256,256,0,256)
tdposB.GetXaxis().SetTitle("mean x position of the cluster")
tdposB.GetYaxis().SetTitle("mean y position of the cluster")

multB = Hist(50,0,50)
multB.GetXaxis().SetTitle("number of hits in the cluster")
multB.GetYaxis().SetTitle("number of clusters")

dthistB = Hist(100,0,100)
dthistB.GetXaxis().SetTitle("time diff between a hit and the min time of the cluster")
dthistB.GetYaxis().SetTitle("number of clusters")

dtime_toaB = Hist2D(100,0,100,230,0,230)
dtime_toaB.GetXaxis().SetTitle("time diff between a hit and the min time of the cluster")
dtime_toaB.GetYaxis().SetTitle("ToT")

cB = ROOT.TCanvas("my4","Clusters B", 1000,1000)
cB.Divide(2,2)

blen = hitsB #readTimepixBufferFromFile() - but this has no definition, so I input the number of entries/data points we have
    
while(b<blen):        # for every buffer position in the buffer?
    cn = 0            # zero the cluster counter for the current buffer position
    cl[cn] = b        # assume the current position has the first hit in the cluster and save it#
    cn+=1             # increment the counter, since we've counted a hit
        
    # this next bit was a for loop in Ken's code
    c=b+1             # start at position c that is one above the current buffer position
    while((c<b+99) and (c<blen)):                  # check all positions up to 99 away, as long as we're still in the buffer
        if(abs(dataB[c][0]- dataB[b][0] < 100)):   # if an event at c is within 100ns of that at the current buffer position
            cl[cn] = c                             # save this position to the cluster
            cn+=1                                  # increment the counter, since we've counted a hit
        else:
            process_clusterB()               # analyze the cluster we found
            b+=cn                           # move on beyond this cluster
            cn = 0                          # reset the counter for hits in the cluster
            cl = [None]*100                 # reset the array of cluster hits
            nc+=1                           # increment the number of clusters found
                
            if(c<b+99 and c<blen):          # if we're still in the loop
                cl[cn] = b                  # assume current position is the first in the new cluster
                cn+=1                       # increment the counter, since we've counted a hit
                
        c+=1 # move to the next position and go through this loop again
    b+=1
    
print(nc)

cB.cd(1)
tdposB.Draw("colz")
cB.cd(2)
multB.Draw()
cB.cd(3)
dthistB.Draw()
cB.cd(4)
dtime_toaB.Draw("colz")
cB.Draw()

16404
